In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import joblib
import pandas as pd
from skimage.color import rgb2lab, lab2rgb

In [3]:
print("Hello World!")

Hello World!


In [ ]:
from ShallowLearn.Indices import mask_land

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Import the ShallowLearn package
import ShallowLearn as sl

In [ ]:
from ShallowLearn import ImageHelper as ih
from ShallowLearn import Transform as tr

In [ ]:
path = "/Volumes/NO NAME"

In [ ]:
models = os.path.join(path, os.listdir(path)[2])
tabular_data = os.path.join(path, os.listdir(path)[3])
cleaned_data = os.path.join(path, os.listdir(path)[0])
honduras = os.path.join(path, os.listdir(path)[1])

In [ ]:
model = joblib.load(os.path.join(models, os.listdir(models)[0]))

In [ ]:
file_list = glob.glob(cleaned_data + '/*6880*.tiff')


In [ ]:
file_list

In [ ]:
image_means = {}
for file in file_list:
    image = ih.load_img(file)
    image_means[file] = np.nanmean(image)

In [ ]:
import re

In [ ]:
#  '/Volumes/NO NAME/Cleaned_Data_Directory/6880_T55LCD_20210720T003709no_transform.tiff',
#  '/Volumes/NO NAME/Cleaned_Data_Directory/6880_T55LCD_20210730T003709no_transform.tiff',
#  '/Volumes/NO NAME/Cleaned_Data_Directory/6880_T55LCD_20220307T003659no_transform.tiff',
#  '/Volumes/NO NAME/Cleaned_Data_Directory/6880_T55LCD_20220406T003659no_transform.tiff',
#  '/Volumes/NO NAME/Cleaned_Data_Directory/6880_T55LCD_20220526T003709no_transform.tiff'
# Regex statement to extract date from above string

def get_date(string):
    return re.findall(r'\d{8}', string)[0]

# Get the dates from the file names
dates = [get_date(file) for file in file_list]

In [ ]:
# convert dates to pd.datetime objects
dates = pd.to_datetime(dates, format='%Y%m%d')

In [ ]:
# matplotlib plot that plots means against date times
plt.plot(dates, list(image_means.values()), 'o')

In [ ]:
land_masks = {}
for file in file_list:
    image = ih.load_img(file)
    land_masks[file] = mask_land(tr.LCE_multi(image), 'B08', 10)
    image_means[file] = np.nanmean(ih.apply_mask(image, land_masks[file]))

In [ ]:
land_masks = {}
image_files_masked = {}
for file in file_list:
    image = ih.load_img(file)
    land_masks[file] = mask_land(tr.LCE_multi(image), 'B08', 10)
    image_means[file] = np.nanmean(ih.apply_mask(image, ~land_masks[file]))
    image_files_masked[file] = ih.apply_mask(image, ~land_masks[file])

In [ ]:
# matplotlib plot that plots means against date times
plt.plot(dates, list(image_means.values()), 'o')

In [ ]:
import joblib

In [ ]:
models

In [ ]:
model_path = os.path.join(models, os.listdir(models)[0])

In [ ]:
model = joblib.load(model_path)

In [ ]:
ih.plot_rgb(image_files_masked[file], plot = True)

In [ ]:
ih.discrete_implot(ih.predict_mask(image_files_masked[file], model_path))

In [ ]:
reefs_only = {}
reef_means = {}
for keys, vals in image_files_masked.items():
    mask = np.expand_dims(ih.predict_mask(image_files_masked[file], model_path, mask_val=4), axis = 2)
    reefs_only[keys] = ih.apply_mask(vals, mask)
    #transform image to lab space
    reefs_only[keys] = rgb2lab(ih.plot_rgb(reefs_only[keys]))
    # lce transform the lab image
    reefs_only[keys][:,:,0] = tr.BCET(reefs_only[keys][:,:,0])
    reef_means[keys] = np.nanmean(reefs_only[keys], axis = (0,1))

In [ ]:
# matplotlib plot that plots means against date times
plt.plot(dates, list(reef_means.values()), 'o')
plt.gca().set_ylim([109.9, 110.05])

In [ ]:
reefs_only = {}
reef_means = {}
for keys, vals in image_files_masked.items():
    mask = np.expand_dims(ih.predict_mask(image_files_masked[file], model_path, mask_val=4), axis = 2)
    reefs_only[keys] = ih.apply_mask(vals, mask)
    #transform image to lab space
    reefs_only[keys] = rgb2lab(ih.plot_rgb(reefs_only[keys]))
    # lce transform the lab image
    try:
        reefs_only[keys][:,:,0] = tr.linear_contrast_enhancement(reefs_only[keys][:,:,0])
        reef_means[keys] = np.nanmean(reefs_only[keys], axis = (0,1))
    except:
        del reefs_only[keys]

In [ ]:
dates = [get_date(file) for file in reefs_only.keys()]

In [ ]:
dates = pd.to_datetime(dates, format='%Y%m%d')

In [ ]:
# extract the first value from the array of means
arr_vals = list(reef_means.values())
arr_vals = [val[0] for val in arr_vals]
# remove the mean from all the values in the array
arr_vals = [val - arr_vals[0] for val in arr_vals]

In [ ]:
# matplotlib plot that plots means against date times
plt.plot(dates, arr_vals, 'o')
# plt.gca().set_ylim([109.9, 110.05])

In [ ]:
# print the outliers in the data of means
for d, v in zip(dates, arr_vals):
    if v < -0.01:
        print(d, v)

In [ ]:
# show the original image data for the outlier
for keys, vals in image_files_masked.items():
    if get_date(keys) == '20190211':
        print(keys)
        ih.plot_rgb(vals, plot = True)
        plt.show()

In [ ]:
ih.predict_mask(image_files_masked[file], model_path, mask_val=4)

In [ ]:
plt.plot(image_means)

In [ ]:
img_test = ih.load_img(file_list[19])


In [ ]:
ih.plot_rgb(tr.BCET_multi(img_test), plot = True)

In [ ]:
ih.plot_lab(tr.BCET_multi(img_test), plot = True)

In [ ]:
ih.plot_histograms(tr.BCET_multi(img_test))

In [ ]:
band_mean = np.mean(tr.BCET_multi(img_test), axis = (0,1))

In [ ]:
img_test = ih.load_img(file_list[25])

In [ ]:
ih.plot_histograms(tr.BCET_multi(img_test))

In [ ]:
ih.plot_rgb(tr.BCET_multi(img_test), plot = True)

In [ ]:
band_mean2 = np.mean((img_test), axis = (0,1))

In [ ]:
lab_img = ih.plot_lab(img_test, plot = False)

In [ ]:
ih.plot_histograms(lab_img)

In [ ]:
lab_img[:,:,0] = tr.linear_contrast_enhancement(lab_img[:,:,0])

In [ ]:
ih.plot_histograms(tr.LCE_multi(lab2rgb(lab_img)))

In [ ]:
plt.imshow(lab2rgb(lab_img))

In [ ]:
ih.plot_histograms(tr.LCE_multi(img_test - (band_median2)), plot = True)

In [ ]:
# Import the data
data = tr.LCE_multi(ih.load_img('/Users/ziad/Documents/GitHub/ShallowLearn/data/T55LCD_20160409T002712no_transform.tiff'))

In [ ]:
ih.plot_rgb(data, plot = True)

In [ ]:
# generate a list of integers up to 500
n = 500
x = np.arange(n)
# randomly disorganise the list
np.random.shuffle(x)

In [ ]:
# function sorting the list
def sort_list(x):
    # create an empty list
    y = []
    # loop over the list
    for i in range(len(x)):
        # append the smallest value of the list
        y.append(np.min(x))
        # remove the smallest value from the list
        x = np.delete(x, np.argmin(x))
    return y
#q what is the o time of the function and the sorting method used
#a o(n^2) and selection sort
#q how can we make it more efficient
#a use a different sorting algorithm
#q what other sorting algorithms do you know
#a bubble sort, quick sort, merge sort, insertion sort, selection sort, heap sort, etc.
#q what would be the fastest


In [ ]:
# function sorting the list using quick sort
def quick_sort_list(x):
    # if the list is empty, return an empty list
    if len(x) == 0:
        return []
    # if the list has only one element, return the list
    if len(x) == 1:
        return x
    # if the list has more than one element, use quick sort
    else:
        # select the pivot
        pivot = x[0]
        # create two empty lists
        left = []
        right = []
        # loop over the list
        for i in range(1, len(x)):
            # if the element is smaller than the pivot, append it to the left list
            if x[i] < pivot:
                left.append(x[i])
            # if the element is larger than the pivot, append it to the right list
            else:
                right.append(x[i])
        # return the sorted left list, the pivot and the sorted right list
        return sort_list(left) + [pivot] + sort_list(right)


In [ ]:
#time the functions
import time
# time the function sort_list
start = time.time()
sort_list(x)
end = time.time()
print('sort_list: ', end - start)
# time the function quick_sort_list
start = time.time()
quick_sort_list(x)
end = time.time()
print('quick_sort_list: ', end - start)


In [ ]:

import random

# generate a list of 1000 random numbers from 1 to 1000000
numbers = random.sample(range(1, 1000001), 1000)


In [ ]:
# time the function sort_list
start = time.time()
sort_list(numbers)
end = time.time()
print('sort_list: ', end - start)
# time the function quick_sort_list
start = time.time()
quick_sort_list(numbers)
end = time.time()
print('quick_sort_list: ', end - start)


In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import rasterio
from rasterio.warp import transform_geom, calculate_default_transform
from shapely.geometry import Point, Polygon


In [ ]:
data

In [ ]:
coral_watch_path = os.path.join(tabular_data, 'coral_watch_ala')

In [ ]:
os.listdir(coral_watch_path) 

In [ ]:
australia_coral_watch = os.path.join(coral_watch_path, 'data.xlsx')

In [ ]:
df = pd.read_excel(australia_coral_watch, sheet_name="CoralWatch Random Survey")
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [ ]:
gdf['Observation date'] = pd.to_datetime(gdf['Observation date'])

In [ ]:
average_data_frame = gdf.groupby(['Observation date', 'Site Name']).mean()['Average.']

In [ ]:
grouped_data = average_data_frame.reset_index()
grouped_data['Observation date'] = pd.to_datetime(grouped_data['Observation date'])
grouped_data = grouped_data.set_index(['Observation date', 'Site Name'])

In [ ]:
# Assuming your DataFrame is named 'grouped_data' with the fixed datetime index
series_data = grouped_data['Average.']
series_data = series_data.reset_index()  # Convert DataFrame to Series
series_data['Observation date'] = pd.to_datetime(series_data['Observation date'])  # Convert to datetime type

# Plotting scatter points of average values versus observation dates
plt.figure(figsize=(15, 10))
plt.scatter(series_data['Observation date'], series_data['Average.'], s=50, marker='o')
# add another dataset to scatter
plt.scatter(dates, arr_vals, s=50, marker='o')
plt.xlabel('Observation date')
plt.ylabel('Average')
plt.title('Observation Date vs Average Australia')
plt.grid(True)
plt.show()

In [ ]:
#linear interpolate the arr_vals
from scipy.interpolate import interp1d


In [ ]:
dates.columns = ['Observation date']

In [ ]:
f = interp1d(dates['Observation date'], arr_vals, kind='linear')

In [ ]:
series_data['Observation date'] = pd.to_datetime(series_data['Observation date'])

In [ ]:
# remove seconds from pd.datetime
series_data['Observation date'] = series_data['Observation date'].dt.floor('d')

In [ ]:
f2 = interp1d(series_data['Observation date'], series_data['Average.'], kind='linear')

In [ ]:
f2(dates['Observation date'])

In [ ]:
# generate a linear regression model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# regress the observation dates against the average values
regression_model = LinearRegression()
regression_model.fit(series_data['Observation date'].values.reshape(-1, 1), series_data['Average.'].values.reshape(-1, 1))

In [ ]:
# predict the average values from the observation dates
y_predicted = regression_model.predict(series_data['Observation date'].values.reshape(-1, 1))